# Cloud Reachability and Latency Forecasting with RIPE Atlas
This project is accesible at: https://github.com/rita-imdea/ripe-userguide

## I) Creating your own customized measurements

In this notebook, we are going to see how to create new measurements. We can freely select the probes and the IP or URL of the destination. We recommend the reader to first take a look at the PDF document that contains the guide for the whole course. 
This notebook is divided in four different steps:

1. We import the required packages and libraries
2. We provide the correct values for the API key, and the list of probes and destinations. 
3. Next, we transform the input data (IDs and URLs) into the correct format. 
4. We create the measurement and we set the parameters 

----------------------------------------------

##### 1. Packages
We start by adding the required packages


In [1]:
import csv # To read from CSV files 

from datetime import datetime # To check the current time
from datetime import timedelta # To compute distance in time

# RIPE Atlas API
from ripe.atlas.cousteau import (
    Ping,
    Traceroute,
    AtlasSource,
    AtlasCreateRequest
)

 #### 2. API  + lists
 Now it is time to include your API Key to be able to run the measurements. You find it in https://atlas.ripe.net/keys/ <br>
 We also provide the file that contains the list of probe's IDs and destination's urls. 
 Note that the example assumes that the lists are given in a <strong>CSV format</strong>.

In [2]:
ATLAS_API_KEY = ""

my_probes_file = "list_id_probes.csv"
my_urls_file   = "list_url_ip_destination.csv"

#### 3. Formatting
Next, we have to transform the input data (CSV files) into the format that is required by the RIPE Atlas API. For that, we read the CSV files row by row, extracting the first column (where the useful information is stored), and then:

   a) For the probes, we save them as a string containing the ids separated by commas.<br>
   b) For the destinations, we save them as a list since we need to run a different experiment for each destination. 

In [3]:
# Putting the source probes in a comma-separated string
with open(my_probes_file,"r") as probes: ### the file with the IDs of the probes you want to use.
    probesReader= csv.reader(probes,delimiter=";") ### Reading CSV file with the probes IDs in the first column of the file.
    next(probes)
    probesList= []
    for i in probesReader:
        probesList.append(i[0]) # Saving the IDs
    probesStr = ",".join(probesList) # join the IDs as a comma-separated string

In [4]:
# As for the probes, we obtain list of destinations that we are targetting. 
with open(my_urls_file,"r") as urls:
    urlsReader = csv.reader(urls,delimiter="\n")
    urlsList = []
    for i in urlsReader:
        urlsList.append(i[0])

We can check that the values have been correctly saved:

In [5]:
print("List of probes' IDs: ", probesStr)
print("List of destinations: ", urlsList)

List of probes' IDs:  1004991,53229,1004997,1004102,20757,1003454,1003158,1003747,1004200,51381,54470,1002914
List of destinations:  ['198.27.190.45', 'ec2.ap-northeast-3.amazonaws.com']


###### 4. Creating measurement

Finally, for each one of the URLs or IP addresses that we are targeting (i.e., that we are considering as a destination), we have to call the API to set and start the experiment. 

   Note that <strong>RIPE Atlas' experiments only allow us to set one destination</strong>. Thus, if we want to do the same measurement for different URLs/IPs, we need to create different measurements with the same parameters. This is easily done in Python with a loop, while in the web interface it would become a tedious task. 

In the following, we first set the parameters of the measurement. We use two different types of measurement: with PING packets (providing information only of the round-trip time), and TRACEROUTE, which provides information about the IP path followed by the packets. 

We can also set the starting time for the measurement and how long it lasts. 

In [6]:
## Now, for each url that we target, we create the experiment.
measurement_dict = {}
for url_i in urlsList: # for each destination
    
    # We set a first measurement with Ping packets, that is repeated every 5 minutes (300 seconds)
    ping = Ping(
                    af=4,
                    target=url_i,
                    interval=300, #run ping every 5 minutes 
                    description="Ping Test"
                )
    
    # We set a measurement with Traceroute protocol, that is repeated every 2 hours (7200 seconds)
    traceroute = Traceroute(
                                af=4,
                                target=url_i,
                                interval=7200, # run traceroute every 2 hours
                                description="Traceroute Test",
                                protocol="ICMP",
                            )
    
    # We set the source probes from our list.
    source = AtlasSource(
                                type="probes",
                                requested=len(probesList),
                                value= probesStr,
                                tags={"exclude": ["system-anchor"]}
                            )
    
    # We create the request for the measurement indicating the starting and finishing time
    atlas_request = AtlasCreateRequest(
                                                start_time=datetime.utcnow()+ timedelta(seconds=300),
                                                stop_time= datetime.utcnow()+ timedelta(seconds=300)+timedelta(days=2),
                                                key=ATLAS_API_KEY,
                                                measurements=[ping, traceroute],
                                                sources=[source, source]
                                        )

    # We make the request trhough the API.
    (is_success, response) = atlas_request.create()
    
    # We can finally check if the measurement has been successfully started. 
    if is_success:
        measurement_dict.update(response) 
    else:
        print(response)
        
    print(measurement_dict)

{'error': {'detail': 'Authentication credentials were not provided.', 'status': 401, 'title': 'Unauthorized', 'code': 104}}
{}
{'error': {'detail': 'Authentication credentials were not provided.', 'status': 401, 'title': 'Unauthorized', 'code': 104}}
{}


When the previous code has finished, if the measurement has been successful, the code will print the <strong>IDs of the measurements</strong>. These IDs are important because they identify the measurement and allow you to later retrieve the data from the database, even when the measurements are still running. 

Accessing public measurements and accessing your own measurements follows the same procedure. Such procedure is explained in the other Jupyter Notebook of the course.   

----------------------------------------------

----------------------------------------------